In [18]:
import os
import glob
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [19]:
FCPXML_BASE_DIR = '..\\..\\..\\CallanSpeech\\static'
OUT_XML_BASE_DIR = '.\\temp'

In [20]:
def parse_time(time_str):
    if '/' in time_str:
        numerator, denominator = time_str[:-1].split('/')
        return round(float(numerator) / float(denominator), 3)
    else:
        return round(float(time_str[:-1]), 3)

In [21]:
class Speaker:
    def __init__(self, name, start, duration, note, image, translation):
        self.name = name
        self.start = start
        self.duration = duration
        self.note = note
        self.image = image
        self.translation = translation
        
class SpeakersCollection:
    def __init__(self):
        self.collection = {}

    def add_speaker(self, id, speaker):
        if id not in self.collection:
            self.collection[id] = {}
        self.collection[id][speaker.name] = speaker
        
    def get_speaker(self, id, name):
        if id in self.collection and name in self.collection[id]:
            return self.collection[id][name]
        else:
            return None

    def get_all_speakers_for_id(self, id):
        if id in self.collection:
            return self.collection[id]
        else:
            return None        
        
speakers = SpeakersCollection()
speaker1 = Speaker('Speaker1', 0.076, 1.2, "What's this?", '../p1.ng', 'Что это?')
speaker2 = Speaker('Speaker2', 1.3, 1.0, 'This is a pen', '../p2.ng', None)
speakers.add_speaker(0, speaker1)
speakers.add_speaker(0, speaker2)

print(vars(speakers.get_speaker(0, 'Speaker1')))
all_speakers_0 = speakers.get_all_speakers_for_id(0)
for name, speaker in all_speakers_0.items():
    print(f'{name}: {vars(speaker)}')


{'name': 'Speaker1', 'start': 0.076, 'duration': 1.2, 'note': "What's this?", 'image': '../p1.ng', 'translation': 'Что это?'}
Speaker1: {'name': 'Speaker1', 'start': 0.076, 'duration': 1.2, 'note': "What's this?", 'image': '../p1.ng', 'translation': 'Что это?'}
Speaker2: {'name': 'Speaker2', 'start': 1.3, 'duration': 1.0, 'note': 'This is a pen', 'image': '../p2.ng', 'translation': None}


In [31]:
def convert_fcpxml_to_xml(fcpxml_file):
    source = None
    start = None
    duration = None
    
    tree = ET.parse(fcpxml_file)
    root = tree.getroot()
    resources = root.find('resources')
    if(resources):
        asset = resources.find('asset')
        if(asset is not None):
            src = asset.attrib['src']
            start = asset.attrib['start']
            duration = asset.attrib['duration']
            
            if((src is None) or (start is None) or (duration is None)):
                print('src or start or duration is None', fcpxml_file)
                return None
            
    start = parse_time(start)
    duration = parse_time(duration)
            
    spine = root.find('.//spine')
    if(spine is None):
        print('spine is None', fcpxml_file)
        return None
    
    asset_clips = spine.findall('asset-clip')
    
    item_id = -1
    speakers = SpeakersCollection()
    
    for asset_clip in asset_clips:
        asset_name, asset_start, asset_duration = asset_clip.attrib['name'], asset_clip.attrib['start'], asset_clip.attrib['duration']
        if((asset_name is None) or (asset_start is None) or (asset_duration is None)):
            print('asset_name or asset_start or asset_duration is None', fcpxml_file)
            return None

        asset_start = parse_time(asset_start)
        asset_duration = parse_time(asset_duration)        
        
        if(asset_name == 'Speaker1'):
            item_id += 1
        elif (asset_name != 'Speaker2'):
            print('bad speaker name', asset_name, asset_start, item_id, fcpxml_file)
            return None
        
        note, translation, image, translation = asset_clip.find('note'), asset_clip.find('translation'), \
                                                asset_clip.find('image'), asset_clip.find('translation')
        
        speaker = Speaker(name=asset_name, start=asset_start, duration=asset_duration, note=note.text, \
                          image=image.text, translation=translation.text)
        speakers.add_speaker(item_id, speaker)
            
    return start, duration, speakers
        

In [39]:
def convert_all():
    good_counter = bad_counter = 0
    items = os.listdir(FCPXML_BASE_DIR)
    for item in items:
        if(item.startswith('Part')):
            part_path = os.path.join(FCPXML_BASE_DIR, item)
            part_path_dirs = os.listdir(part_path)
            for lesson_dir in part_path_dirs:
                full_lesson_dir = os.path.join(part_path, lesson_dir)
                if (os.path.isdir(full_lesson_dir) and lesson_dir.startswith('Lesson')):
                    for fcpxml_file in glob.glob(os.path.join(full_lesson_dir, '*.fcpxml')):
                        general_start, general_duration, speakers = convert_fcpxml_to_xml(fcpxml_file)
                        if(speakers is not None):
                            good_counter += 1                               
                            output_root = ET.Element("items", start=str(general_start), duration=str(general_duration))
                            loc_ru_root = ET.Element("items")
                            for id, speakers in speakers.collection.items():
                                for name, speaker in speakers.items():
                                    output_item = ET.SubElement(output_root, "item", id=str(id), \
                                                                start=str(speaker.start), duration=str(speaker.duration))
                                    person = ET.SubElement(output_item, "person")
                                    person.text = 'Teacher' if speaker.name == 'Speaker1' else 'Student'
                                    note = ET.SubElement(output_item, "text")
                                    note.text = speaker.note
                                    image = ET.SubElement(output_item, "image")
                                    image.text = speaker.image
                                    
                                    loc_ru_output_item = ET.SubElement(loc_ru_root, "item", id=str(id))
                                    translation = ET.SubElement(loc_ru_output_item, "translation")
                                    translation.text = speaker.translation
                                                            
                            short_name = os.path.basename(fcpxml_file).split('.', 1)[0]
                            output_xml = os.path.join(OUT_XML_BASE_DIR, short_name + '.xml')
                            
                            xml_output_string = minidom.parseString(ET.tostring(output_root)).toprettyxml(indent=" ")
                            with open(output_xml, "w", encoding='utf-8') as f:
                                f.write(xml_output_string)
                                
                            output_loc_ru_xml = os.path.join(OUT_XML_BASE_DIR, short_name + '_translations_ru.xml')
                            xml_output_string = minidom.parseString(ET.tostring(loc_ru_root)).toprettyxml(indent=" ")
                            with open(output_loc_ru_xml, "w", encoding='utf-8') as f:
                                f.write(xml_output_string)
                            
                            #return
                        else:
                            bad_counter += 1

                            
    print(good_counter, bad_counter)
                

In [40]:
convert_all()

59 0
